# MNIST数据处理

1. MNIST 是NIST的子一个子集。
2. 60000张图片作为训练集，10000张作为验证集。
3. $28x28$图片

## 看具体代码

tensorflow 具有处理MNIST数据的程序包。具体运用如下

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data


# 如果对应文件夹没有数据，则会下载。
mnist = input_data.read_data_sets("./data/", one_hot=True)

# 其中的one_hot参数就是说一个label中只有一个1.
# 比如(0,0,0,1)就是one_hot，而(0,1,1,0)就不是one_hot

# 打印训练数据大小
print("Training data size ", mnist.train.num_examples)

# 打印验证数据大小
print("Validation data size ", mnist.validation.num_examples)

# 打印测试数据大小
print("Testing data size", mnist.test.num_examples)

# print an example training data
print("An example training data:", mnist.train.images[0])

# print example training data label
print("Example training data label", mnist.train.labels[0])

#可以设置取数据的batch大小

batch_size = 10
xs, ys = mnist.train.next_batch(batch_size)

print("X Shape", xs.shape)
print("Y Shape", ys.shape)


# 训练代码(一个完整的示例)

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# MNIST data set related constant

INPUT_NODE = 784
OUTPUT_NODE = 10

# neural network parameters
LAYER1_NODE = 500
BATCH_SIZE = 100

LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99

REGULARAZTION_RATE = 0.0001
TRAINING_STEPS = 30000
MOVING_AVERAGE_DECAY = 0.99

# auxiliary functions


def inference(input_tensor, avg_class, weight1, biases1,
              weights2, biases2):
    if avg_class is None:
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weight1)+biases1)
        return tf.matmul(layer1, weights2)+biases2
    else:
        layer1 = tf.nn.relu(
            tf.matmul(input_tensor, avg_class.average(weight1)) +
            avg_class.average(biases1))
        return (tf.matmul(layer1, avg_class.average(weights2)) +
                avg_class.average(biases2))

# The training process


def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name="x-input")
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')

    #tf.truncated_normal 如果取到的值超过两个标准差，则重新pick
    weights1 = tf.Variable(
        tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))

    weights2 = tf.Variable(
        tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))

    # 生成网络,但是不使用滑动平均，所以传入None
    y = inference(x, None, weights1, biases1, weights2, biases2)

    # 定义存储训练轮次的变量。这个变量不需要计算滑动平均。所以设置为(trainable=False)。
    # 训练轮次一般设置为不可训练参数
    global_step = tf.Variable(0, trainable=False)

    # 定义滑动平均类
    variable_averages = tf.train.ExponentialMovingAverage(
        MOVING_AVERAGE_DECAY, global_step)

    # 对训练图上可训练的参数所有全部应用滑动平均,其中tf.trainable_variables返回的是
    # GraphKeys.TRAINABLE_VARIABLES中的元素，也就是那些没有被设置trainable=False的
    # 参数。
    variables_averages_op = variable_averages.apply(
        tf.trainable_variables())

    # 生成网络，这次使用滑动平均,因为上一步是应用滑动平均到变量上，本质是计算一个影子变量。
    # 要想使用这个影子变量，必须认为应用它，也就是这里设置variable_averages的原因。
    average_y = inference(
        x, variable_averages, weights1, biases1, weights2, biases2)

    # 计算交叉熵,tf.argmax 会给出向量中最大值的坐标位置。(2,3,4,5,1) -> 3
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=y, labels=tf.argmax(y_, 1))

    # 计算一个batch中交叉熵的均值
    cross_entropy_mean = tf.reduce_mean(cross_entropy)

    # 计算损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARAZTION_RATE)
    regularaztion = regularizer(weights1) + regularizer(weights2)
    loss = cross_entropy_mean + regularaztion

    # 设置指数衰减的学习率。
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE,
        LEARNING_RATE_DECAY,
        staircase=True)

    # 优化损失函数
    train_step = tf.train.GradientDescentOptimizer(
        learning_rate).minimize(loss, global_step=global_step)

    # 在训练神经网络模型时，每过一遍数据既需要通过反向传播来更新神经网络中的参数，
    # 又要更新每一个参数的滑动平均值。为了一次完成多个操作， TensorFlow 提供了
    # tf.control_dependencies和tf.group两种机制。 下面两行程序和
    # train_op = tf.group(train_step, variables_averages_op)是等价的。
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')

    # 计算正确率
    # 在验证数据上使用滑动平均，而不是在训练数据上使用滑动平均。
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        validate_feed = {x: mnist.validation.images,
                         y_: mnist.validation.labels}

        test_feed = {x: mnist.test.images, y_: mnist.test.labels}

        # 循环的训练神经网络。
        for i in range(TRAINING_STEPS):
            if i % 1000 == 0:
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training step(s), validation accuracy using average model is %g "
                      % (i, validate_acc))

            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict={x: xs, y_: ys})

        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print(("After %d training step(s), test accuracy using average model is %g"
               % (TRAINING_STEPS, test_acc)))


def main(argv=None):
    mnist = input_data.read_data_sets("./data/",
                                      one_hot=True)
    train(mnist)


if __name__ == '__main__':
    main()


## 如何验证数据

1. 交叉验证的方式效果比较好。
2. 但是面对海量数据的时候，一般采用上面的验证数据集的形式来评估。
3. 在验证数据上使用滑动平均，而不是在训练数据上使用滑动平均。

原因:对神经网络边的权重 weights 使用滑动平均，得到对应的影子变量 shadow_weights。在训练过程仍然使用原来不带滑动平均的权重 weights，不然无法得到 weights 下一步更新的值，又怎么求下一步 weights 的影子变量 shadow_weights。之后在测试过程中使用 shadow_weights 来代替 weights 作为神经网络边的权重，这样在测试数据上效果更好。因为 shadow_weights 的更新更加平滑，对于随机梯度下降而言，更平滑的更新说明不会偏离最优点很远；对于梯度下降 batch gradient decent，我感觉影子变量作用不大，因为梯度下降的方向已经是最优的了，loss 一定减小；对于 mini-batch gradient decent，可以尝试滑动平均，毕竟 mini-batch gradient decent 对参数的更新也存在抖动。

**验证数据集合测试数据集要有一定的一致性。模型才能有更好表现。**

## 不同模型效果比较

### 回归第四章的5种优化方法

1. 激活函数。
2. 多层网络。
3. 学习率指数衰减。
4. 正则化的损失函数。
5. 滑动平均。

### 损失函数比单独交叉熵，更能挖掘数据的潜力。


In [ ]:
import tensorflow as tf
# cross-entropy

train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(
    cross_entropy_mean, global_step = global_step)

# cross-entropy and regularization

loss = cross_entropy_mean + regularaztion
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(
    cross_entropy_mean, global_step = global_step)

![picture](regu.png)

# 变量管理

## 创建变量


In [ ]:
tf.get_variable
tf.variable_scope

# 等价问题

v = tf.get_variable("v", shape=[1], initializer=tf.constant_initializer(1.0))
v = tf.Variable(tf.constant(1.0, shape=[1]), name="v")

tensorflow的初始化函数

1. tf.constant_initializer
2. tf.random_normal_initializer
3. tf.tnmcated_norrηal_imtializer
4. tf.random_uniform_initializer
5. tf.uniform_unit_sealing_initializer
6. tf.zeros_initializer
7. tf.ones_initializer

### 通过tf.variable_scope指定上下文环境


In [ ]:

import tensorflow as tf

with tf.variable_scope("foo"):
    v = tf.get_variable("v", [1], initializer=tf.constant_initializer(1.0))

#因为在命名v存在了，所以以下代码将会报错:
#with tf.variable_scope("foo"):
#   v = tf.get_variable("v", [1]) # Error!!!

with tf.variable_scope("foo", reuse=True):
    v1 = tf.get_variable("v", [1])
    print(v == v1)

# 命名空间var中没有创建变量v，所以以下代码将会Error
#with tf.variable_scope("bar", reuse=True):
#   v = tf.get_variable("v", [1])

###  嵌套上下文管理器中的reuse参数的使用


In [ ]:
with tf.variable_scope("root"):
    print(tf.get_variable_scope().reuse)
    with tf.variable_scope("foo", reuse=True):
        print(tf.get_variable_scope().reuse)
        with tf.variable_scope("bar"):
            print(tf.get_variable_scope().reuse)
    print(tf.get_variable_scope().reuse)

### 通过variable_scope来管理变量

In [ ]:
import tensorflow as tf

v1 = tf.get_variable("v", [1])
print(v1.name)

with tf.variable_scope("foo",reuse=True):
    v2 = tf.get_variable("v", [1])
    print(v2.name)

with tf.variable_scope("foo"):
    with tf.variable_scope("bar"):
        v3 = tf.get_variable("v", [1])
        print(v3.name)

v4 = tf.get_variable("v1", [1])
print(v4.name)

### 我们可以通过变量的名称来获取变量

In [ ]:
with tf.variable_scope("",reuse=True):
    v5 = tf.get_variable("foo/bar/v", [1])
    print v5 == v3
    v6 = tf.get_variable("v1", [1])
    print v6 == v4

# Tensorflow 模型持久化


In [ ]:
v1 = tf.Variable(tf.random_normal([1], stddev=1, seed=1))
v2 = tf.Variable(tf.random_normal([1], stddev=1, seed=1))
result = v1 + v2

init_op = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init_op)
    saver.save(sess, "Saved_model/model.ckpt")

这个目录下回出现三个文件

1. model.ckpt.meta 计算图结构
2. model.ckpt 每一个变量的值
3. checkpoint

## 加载

In [ ]:
v1 = tf.Variable(tf.random_normal([1], stddev=1, seed=1))
v2 = tf.Variable(tf.random_normal([1], stddev=1, seed=1))
result = v1 + v2

saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess, "Saved_model/model.ckpt")
    print sess.run(result)

1. 两段代码的区别，第二段代码没有初始化过程。
2. 如果不想重复定义图上的计算。可以直接加载已经持久化固化的图。

In [ ]:
# 直接加载持久化的图。因为之前没有导出v3，所以这里会报错

saver = tf.train.import_meta_graph("Saved_model/model.ckpt.meta")
v3 = tf.Variable(tf.random_normal([1], stddev=1, seed=1))

with tf.Session() as sess:
    saver.restore(sess, "Saved_model/model.ckpt")
    print sess.run(v1)
    print sess.run(v2)
    print sess.run(v3)

In [ ]:
import tensorflow as tf

saver = tf.train.import_meta_graph("path/model.ckpt.meta")

with tf.Session() as sess:
    saver.restore(sess, "path/mode.ckpt")
    print(sess.run(tf.get_default_graph().get_tensor_by_name("add:0")))

## 变量重命名

In [ ]:
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name = "other-v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name = "other-v2")
saver = tf.train.Saver({"v1": v1, "v2": v2})

### 通过重命名读取滑动平均

In [ ]:
v = tf.Variables(0, dtype=tf.float32, name="v")
saver=tf.train.Saver({"v/ExponentialMovingAverage":v})
with tf.Session() as sess:
    saver.restore(sess, "path/mode.ckpt")
    print(sess.run(v))

In [ ]:
import tensorflow as tf
v = tf.Variable(0, dtype=tf.float32, name="v")
ema = tf.train.ExponentialMovingAverage(0.99)
print ema.variables_to_restore() ## 这个函数，可以自动定义前面代码定义的变量的滑动平均重命名字典。

saver = tf.train.Saver({"v/ExponentialMovingAverage": v})
with tf.Session() as sess:
    saver.restore(sess, "Saved_model/model2.ckpt")
    print sess.run(v)

### 使用Saver保存，会报错所有信息。有时候不需要

In [ ]:
import tensorflow as tf
from tensforflow.python.framework import graph_util

v1 = tf.Variable(tf.constant(1.0, shape=[1]), name="v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name="v2")

result = v1 + v2

init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)
    graph_def = tf.get_default_graph().as_graph_def()

    output_graph_def = graph_util.convert_variables_to_constants(
        sess, graph_def, ["add"])
    with tf.gfile.GFile("path/combined_mode.pb", "wb") as f:
        f.write(output_graph_def.SerializeToString())

## add:0 其中add是计算节点的名字，而0表示是它的第一个输出。

### 完成迁移学习


In [ ]:
import tensorflow as tf
from tensorflow.python.platform import gfile

with tf.Session() as sess:
    mode_filename = "path/combined_mode.pb"
    with gfile.FastGFile(model_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromSTring(f.read())

        result = tf.import_graph_def(graph_def, return_elements=["add:0"])
        print(sess.run(result))

# 重构代码

为了让程序结构更清晰，代码更为整洁易读。把程序拆解为三个部分

1. mnist_inference.py
2. mnist_train.py
3. mnist_eval.py

## mnist_inference.py

In [ ]:
import tensorflow as tf

# 定义神经网络结构相关的参数。
INPUT_NODE = 784
OUTPUT_NODE = 10
LAYER1_NODE = 500

# 通过tf.get_variable函数来获取变量。
def get_weight_variable(shape, regularizer):
    weights = tf.get_variable("weights", shape, initializer=tf.truncated_normal_initializer(stddev=0.1))
    if regularizer != None: tf.add_to_collection('losses', regularizer(weights))
    return weights

# 定义神经网络的前向传播过程。

def inference(input_tensor, regularizer):
    with tf.variable_scope('layer1'):

        weights = get_weight_variable([INPUT_NODE, LAYER1_NODE], regularizer)
        biases = tf.get_variable("biases", [LAYER1_NODE], initializer=tf.constant_initializer(0.0))
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)


    with tf.variable_scope('layer2'):
        weights = get_weight_variable([LAYER1_NODE, OUTPUT_NODE], regularizer)
        biases = tf.get_variable("biases", [OUTPUT_NODE], initializer=tf.constant_initializer(0.0))
        layer2 = tf.matmul(layer1, weights) + biases

    return layer2

## mnist_train.py


In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import mnist_inference
import os

# 定义神经网络结构相关的参数
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 30000
MOVING_AVERAGE_DECAY = 0.99
MODEL_SAVE_PATH = "MNIST_model/"
MODEL_NAME = "mnist_model"

# 定义训练过程

def train(mnist):
    # 定义输入输出placeholder。
    x = tf.placeholder(tf.float32, [None, mnist_inference.INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, mnist_inference.OUTPUT_NODE], name='y-input')

    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    y = mnist_inference.inference(x, regularizer)
    global_step = tf.Variable(0, trainable=False)

    # 定义损失函数、学习率、滑动平均操作以及训练过程。
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE, LEARNING_RATE_DECAY,
        staircase=True)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')

    # 初始化TensorFlow持久化类。
    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        for i in range(TRAINING_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: xs, y_: ys})
            if i % 1000 == 0:
                print("After %d training step(s), loss on training batch is %g." % (step, loss_value))
                saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=global_step)

# 主程序入口

def main(argv=None):
    mnist = input_data.read_data_sets("./data", one_hot=True)
    train(mnist)

if __name__ == '__main__':
    main()

## mnist_eval.py

In [ ]:
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import mnist_inference
import mnist_train

# 每10秒加载一次最新的模型
EVAL_INTERVAL_SECS = 10

def evaluate(mnist):
    with tf.Graph().as_default() as g:
        x = tf.placeholder(tf.float32, [None, mnist_inference.INPUT_NODE], name='x-input')
        y_ = tf.placeholder(tf.float32, [None, mnist_inference.OUTPUT_NODE], name='y-input')
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}

        y = mnist_inference.inference(x, None)
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        variable_averages = tf.train.ExponentialMovingAverage(mnist_train.MOVING_AVERAGE_DECAY)
        variables_to_restore = variable_averages.variables_to_restore()
        saver = tf.train.Saver(variables_to_restore)

        while True:
            with tf.Session() as sess:
                ckpt = tf.train.get_checkpoint_state(mnist_train.MODEL_SAVE_PATH)
                if ckpt and ckpt.model_checkpoint_path:
                    saver.restore(sess, ckpt.model_checkpoint_path)
                    global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                    accuracy_score = sess.run(accuracy, feed_dict=validate_feed)
                    print("After %s training step(s), validation accuracy = %g" % (global_step, accuracy_score))
                else:
                    print('No checkpoint file found')
                    return
            time.sleep(EVAL_INTERVAL_SECS)

# 主程序
def main(argv=None):
    mnist = input_data.read_data_sets("./data", one_hot=True)
    evaluate(mnist)

if __name__ == '__main__':
    main()

# 遗留问题

